In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')

In [2]:
# Find the url for 10 pages
base_url="https://www.imdb.com/search/title/?title_type=feature\
&release_date=1930-01-01,2020-12-31\
&countries=us\
&languages=en&count=100"

url_list=[base_url]
for i in range(5001,25001,100):
    url = base_url+ "&start=" + str(i) + "&ref_=adv_nxt"
    url_list.append(url)

In [3]:
base_url

'https://www.imdb.com/search/title/?title_type=feature&release_date=1930-01-01,2020-12-31&countries=us&languages=en&count=100'

In [4]:
len(url_list)

201

In [5]:
# Create soup_list (13 soup for 13 pages)
def get_soup(url):
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    return soup

In [6]:
soup_list=[]
for url in url_list:
    soup_list.append(get_soup(url))

In [7]:
len(soup_list)

201

In [8]:
# Create movie_div_list (2000 movies, 100 movies a page)
# sort by popularity
movie_div_list=[]
for soup in soup_list:
    for movie in soup.find_all('div',class_="lister-item-content"):
        movie_div_list.append(movie)

In [9]:
len(movie_div_list)

20100

In [10]:
# Create movie_url_list (individual webpage for each movie)

movie_url_list=[]
movie_base_url = "https://www.imdb.com/"

for soup in soup_list:
    divs= [div for div in soup.find_all('div', class_="lister-item-content")]

    for i in range(100):
        movie_url = divs[i].find('a').get('href')
        movie_url_list.append(movie_base_url + movie_url)

In [11]:
len(movie_url_list)

20100

In [12]:
# Create soup_list for the movies

movie_soup_list=[]
for movie_url in movie_url_list[:100]:
    movie_soup = get_soup(movie_url)
    movie_soup_list.append(movie_soup)

In [13]:
len(movie_soup_list)

100

In [14]:
for movie_url in movie_url_list[100:200]:
    movie_soup = get_soup(movie_url)
    movie_soup_list.append(movie_soup)

In [15]:
len(movie_soup_list)

200

In [16]:
for movie_url in movie_url_list[200:500]:
    movie_soup = get_soup(movie_url)
    movie_soup_list.append(movie_soup)

In [17]:
len(movie_soup_list)

500

In [18]:
for movie_url in movie_url_list[500:800]:
    movie_soup = get_soup(movie_url)
    movie_soup_list.append(movie_soup)

In [19]:
len(movie_soup_list)

800

In [20]:
for movie_url in movie_url_list[800:]:
    movie_soup = get_soup(movie_url)
    movie_soup_list.append(movie_soup)

KeyboardInterrupt: 

In [21]:
len(movie_soup_list)

14531

In [22]:
# get detailed data from individual movie webpage
# fixed-- why it's all nan??
# fixed -- runtime

def get_movie_detail(soup):
    
    headers=['movie_title', 'rating', 'vote', 'certificate', 'genre', 'release_date', 'metascore', 'keywords','budget',\
            'opening_weekend_usa','gross_usa','gross_world','runtime',\
             'distributor','language','country',\
             'director','writer','star',\
             'link_d','link_w','link_s']
    
    # find movie title
    title = " ".join(soup.find('h1').text.split()[:-1]) ## problem?
    
    # find rating
#     rating = float(soup.find('span',attrs={'itemprop':'ratingValue'}).text)
    if soup.find('span',attrs={'itemprop':'ratingValue'}) is not None:
        rating = float(soup.find('span',attrs={'itemprop':'ratingValue'}).text)
    else:
        rating = np.nan
    
    # find vote (rating count)
#     vote = int(soup.find('span',attrs={'itemprop':'ratingCount'}).text.replace(',',''))
    if soup.find('span',attrs={'itemprop':'ratingCount'}) is not None:
        vote = int(soup.find('span',attrs={'itemprop':'ratingCount'}).text.replace(',',''))
    else:
        vote = np.nan
    
    # find content rating
#     certificate = soup.find('div', class_="subtext").text.split()[0]
    if soup.find('div', class_="subtext") is not None:
        certificate = soup.find('div', class_="subtext").text.split()[0]
    else:
        certificate = np.nan
    
    # find list of genre
    genre_list=[]
    try:
        for genres in soup.find('div', class_="subtext").find_all('a')[:-1]:
            genre_list.append(genres.text)
    except:
        pass
                 
        
    # find release date
    date = np.nan
    try:
                 
        date_pre = soup.find('div', class_="subtext").find_all('a')[-1].text.split('(')[0]
        date = pd.to_datetime(date_pre) ## why is it Timestamp? format ='%d-%B-%Y'
                 
    except:
        pass
    
    # find metascorre
    if soup.find('div',class_="metacriticScore score_favorable titleReviewBarSubItem") is not None:
        meta = int(soup.find('div',class_="metacriticScore score_favorable titleReviewBarSubItem").text.strip('\n'))
    else:
        meta = np.nan
        
        
    # find plot keywords
    keyword_list=[]
    try:
        for keywords in soup.find('div', class_="article", id="titleStoryLine").\
        find('div', class_="see-more inline canwrap").find_all('a')[:-1]:
            keyword_list.append(keywords.text.strip(' '))
    except:
        pass
    
    
    # find budget, opening weekend USA, gross USA, cumulative worldwide gross, distributor
    # assign default value:
    budget, opening, gross_usa, gross_cw, distributor = np.nan, np.nan, np.nan, np.nan, np.nan
    for line in soup.find('div', class_="article", id="titleDetails").find_all('h4'):        
        if "Budget:" in line:
            budget = int(''.join(s for s in line.next_sibling if s.isdigit()))
        if "Opening Weekend USA:" in line:
            opening = int(''.join(s for s in line.next_sibling if s.isdigit()))
        if "Gross USA:" in line:
            gross_usa = int(''.join(s for s in line.next_sibling if s.isdigit()))
        if "Cumulative Worldwide Gross:" in line:
            gross_cw = int(''.join(s for s in line.next_sibling if s.isdigit()))
        if "Production Co:" in line:
            distributor = line.findNext().text.replace(' ','')


    # find runtime
    runtime = np.nan
    try:
        runtime = int(soup.find_all('time')[-1].text.strip(' min'))
    except:
        pass

    # find director
    director= np.nan
    director = soup.find('div',class_="credit_summary_item").find('a').text
    link_d = soup.find('div',class_="credit_summary_item").find('a').get('href')
    
    # find writer
    writer = np.nan
    writer_line = soup.find_all('div',class_="credit_summary_item")[1].find_all('a')
    link_w = [w.get('href') for w in writer_line]
    writer = [w.text for w in writer_line]
    if '1 more credit' in writer:
        writer.remove('1 more credit')
        link_w.pop()
        
    # find star
    star = np.nan
    star_line = soup.find_all('div',class_="credit_summary_item")[-1].find_all('a')
    link_s = [s.get('href') for s in star_line]
    star = [s.text for s in star_line]
    if 'See full cast & crew' in star:
        star.remove('See full cast & crew')
        link_s.pop()
    
    
    
    # find language
    language= np.nan
    t= []
    matching = []
    for div in soup.find('div', class_="article", id="titleDetails").find_all('div'):
        t.append(div.text.replace('\n','').replace(' ',''))
    
    matching = [s for s in t if 'Language:' in s]
    language = matching[0].replace(':',' ').replace('|',' ').split(' ')[1:]
    
    
    # find country
    country= np.nan
    t= []
    matching = []
    for div in soup.find('div', class_="article", id="titleDetails").find_all('div'):
        t.append(div.text.replace('\n','').replace(' ',''))
    
    matching = [s for s in t if 'Country:' in s]
    country = matching[0].replace(':',' ').replace('|',' ').split(' ')[1:]
    
    
    
    movie_dict = dict(zip(headers, [title,
                                    rating,
                                    vote,
                                   certificate,
                                   genre_list,
                                   date,
                                   meta,
                                   keyword_list,
                                   budget,
                                   opening,
                                   gross_usa,
                                   gross_cw,
                                   runtime,
                                    distributor,
                                    language,
                                    country,
                                   director,
                                    writer,
                                   star,
                                   link_d,
                                    link_w,
                                    link_s,
                                   ]))
    
    return movie_dict

In [23]:
# Create list of movie_dict
movie_detail_data=[]
for movie_soup in movie_soup_list:
    movie = get_movie_detail(movie_soup)
    movie_detail_data.append(movie)

IndexError: list index out of range

In [24]:
len(movie_detail_data)

3212

In [25]:
for movie_soup in movie_soup_list[3213:]:
    movie = get_movie_detail(movie_soup)
    movie_detail_data.append(movie)

In [26]:
len(movie_detail_data)

14530

In [ ]:
movie_soup = movie_soup_list[261]
movie_url_list[261]

In [ ]:
get_movie_detail(movie_soup)

In [37]:
# Create pd DataFrame of 2000 movies!
movie_df2 = pd.DataFrame(movie_detail_data)
movie_df2.sort_values('gross_world',ascending=False)

,movie_title,rating,vote,certificate,genre,release_date,metascore,keywords,budget,opening_weekend_usa,...,runtime,distributor,language,country,director,writer,star,link_d,link_w,link_s
6217,Avengers: Endgame,8.4,804299.0,PG-13,"[Action, Adventure, Drama]",2019-04-26,78.0,"[time travel, superhero, cosmic, super villain...",356000000.0,357115007.0,...,181.0,MarvelStudios,"[English, Japanese, Xhosa, German]",[USA],Anthony Russo,"[Christopher Markus, Stephen McFeely, 14 more ...","[Robert Downey Jr., Chris Evans, Mark Ruffalo]",/name/nm0751577/,"[/name/nm1321655/, /name/nm1321656/, fullcredi...","[/name/nm0000375/, /name/nm0262635/, /name/nm0..."
13217,Avengers: Endgame,8.4,804323.0,PG-13,"[Action, Adventure, Drama]",2019-04-26,78.0,"[time travel, superhero, cosmic, super villain...",356000000.0,357115007.0,...,181.0,MarvelStudios,"[English, Japanese, Xhosa, German]",[USA],Anthony Russo,"[Christopher Markus, Stephen McFeely, 14 more ...","[Robert Downey Jr., Chris Evans, Mark Ruffalo]",/name/nm0751577/,"[/name/nm1321655/, /name/nm1321656/, fullcredi...","[/name/nm0000375/, /name/nm0262635/, /name/nm0..."
5317,Avengers: Endgame,8.4,804299.0,PG-13,"[Action, Adventure, Drama]",2019-04-26,78.0,"[time travel, superhero, cosmic, super villain...",356000000.0,357115007.0,...,181.0,MarvelStudios,"[English, Japanese, Xhosa, German]",[USA],Anthony Russo,"[Christopher Markus, Stephen McFeely, 14 more ...","[Robert Downey Jr., Chris Evans, Mark Ruffalo]",/name/nm0751577/,"[/name/nm1321655/, /name/nm1321656/, fullcredi...","[/name/nm0000375/, /name/nm0262635/, /name/nm0..."
12417,Avengers: Endgame,8.4,804323.0,PG-13,"[Action, Adventure, Drama]",2019-04-26,78.0,"[time travel, superhero, cosmic, super villain...",356000000.0,357115007.0,...,181.0,MarvelStudios,"[English, Japanese, Xhosa, German]",[USA],Anthony Russo,"[Christopher Markus, Stephen McFeely, 14 more ...","[Robert Downey Jr., Chris Evans, Mark Ruffalo]",/name/nm0751577/,"[/name/nm1321655/, /name/nm1321656/, fullcredi...","[/name/nm0000375/, /name/nm0262635/, /name/nm0..."
5217,Avengers: Endgame,8.4,804299.0,PG-13,"[Action, Adventure, Drama]",2019-04-26,78.0,"[time travel, superhero, cosmic, super villain...",356000000.0,357115007.0,...,181.0,MarvelStudios,"[English, Japanese, Xhosa, German]",[USA],Anthony Russo,"[Christopher Markus, Stephen McFeely, 14 more ...","[Robert Downey Jr., Chris Evans, Mark Ruffalo]",/name/nm0751577/,"[/name/nm1321655/, /name/nm1321656/, fullcredi...","[/name/nm0000375/, /name/nm0262635/, /name/nm0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14499,Pieces of a Woman,7.1,16358.0,R,[Drama],2021-01-07,66.0,"[female frontal nudity, female nudity, male fr...",NaN,NaN,...,126.0,BRONStudios,[English],"[Canada, Hungary, USA]",Kornél Mundruczó,[Kata Wéber],"[Vanessa Kirby, Shia LaBeouf, Ellen Burstyn]",/name/nm0610960/,[/name/nm0970537/],"[/name/nm3948952/, /name/nm0479471/, /name/nm0..."
14506,We Can Be Heroes,4.7,8192.0,PG,"[Action, Comedy, Drama]",2020-12-25,NaN,"[non comic book superhero, female protagonist,...",NaN,NaN,...,100.0,DoubleRProductions(II),[English],[USA],Robert Rodriguez,[Robert Rodriguez],"[YaYa Gosselin, Pedro Pascal, Priyanka Chopra]",/name/nm0001675/,[/name/nm0001675/],"[/name/nm8533749/, /name/nm0050959/, /name/nm1..."
14508,One Night in Miami,7.3,6291.0,R,[Drama],2021-01-08,83.0,"[inspired by true events, reference to elijah ...",NaN,NaN,...,114.0,ABKCOFilms,[English],[USA],Regina King,"[Kemp Powers, Kemp Powers]","[Kingsley Ben-Adir, Eli Goree, Aldis Hodge]",/name/nm0005093/,"[/name/nm5358492/, /name/nm5358492/]","[/name/nm4667984/, /name/nm2345856/, /name/nm0..."
14513,Nomadland,7.7,8233.0,R,[Drama],2021-02-19,96.0,"[title directed by female, title written by fe...",NaN,NaN,...,108.0,CorCordiumProductions,[English],"[USA, Germany]",Chloé Zhao,"[Jessica Bruder, Chloé Zhao]","[Frances McDormand, David Strathairn, Linda May]",/name/nm2125482/,"[/name/nm9572798/, /name/nm2125482/]","[/name/nm000053

In [38]:
# Save the data to pickle file
movie_df2.to_pickle('../dump/movie_data_general2')

### Combine with movie_data_genera1

In [39]:
# Load data from dump
movie_df1 = pd.read_pickle('../dump/movie_data_general')

In [40]:
movie_df = pd.concat([movie_df1,movie_df2],axis=0)

In [41]:
movie_df

,movie_title,rating,vote,certificate,genre,release_date,metascore,keywords,budget,opening_weekend_usa,...,runtime,distributor,language,country,director,writer,star,link_d,link_w,link_s
0,Pieces of a Woman,7.1,16183.0,R,[Drama],2021-01-07,66.0,"[female frontal nudity, female nudity, male fr...",NaN,NaN,...,126.0,BRONStudios,[English],"[Canada, Hungary, USA]",Kornél Mundruczó,[Kata Wéber],"[Vanessa Kirby, Shia LaBeouf, Ellen Burstyn]",/name/nm0610960/,[/name/nm0970537/],"[/name/nm3948952/, /name/nm0479471/, /name/nm0..."
1,Soul,8.1,141473.0,PG,"[Animation, Adventure, Comedy]",2020-12-25,83.0,"[purpose, soul, lost soul, cat, afterlife]",NaN,NaN,...,100.0,WaltDisneyPictures,[English],[USA],Pete Docter,"[Pete Docter, Mike Jones]","[Jamie Foxx, Tina Fey, Graham Norton]",/name/nm0230032/,"[/name/nm0230032/, /name/nm0428873/]","[/name/nm0004937/, /name/nm0275486/, /name/nm0..."
2,Wonder Woman 1984,5.4,138578.0,PG-13,"[Action, Adventure, Fantasy]",2020-12-25,NaN,"[princess, sequel, dc extended universe, actio...",NaN,NaN,...,151.0,AtlasEntertainment,[English],"[USA, UK, Spain]",Patty Jenkins,"[Patty Jenkins, Geoff Johns, 4 more credits]","[Gal Gadot, Chris Pine, Kristen Wiig]",/name/nm0420941/,"[/name/nm0420941/, /name/nm0424315/, fullcredi...","[/name/nm2933757/, /name/nm1517976/, /name/nm1..."
3,Tenet,7.5,272191.0,PG-13,"[Action, Sci-Fi, Thriller]",2020-09-03,69.0,"[written by director, car chase, time paradox,...",205000000.0,20200000.0,...,150.0,WarnerBros.,"[English, Russian, Ukrainian, Estonian, Norweg...","[UK, USA]",Christopher Nolan,[Christopher Nolan],"[John David Washington, Robert Pattinson, Eliz...",/name/nm0634240/,[/name/nm0634240/],"[/name/nm0913475/, /name/nm1500155/, /name/nm4..."
4,Promising Young Woman,7.5,15211.0,R,"[Crime, Drama, Thriller]",2020-12-25,72.0,"[revenge, rape and revenge, female protagonist...",NaN,719305.0,...,113.0,FilmNationEntertainment,[English],"[UK, USA]",Emerald Fennell,[Emerald Fennell],"[Carey Mulligan, Bo Burnham, Alison Brie]",/name/nm2193504/,[/name/nm2193504/],"[/name/nm1659547/, /name/nm3102998/, /name/nm1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14525,Once Upon a Time... In Hollywood,7.6,547218.0,R,"[Comedy, Drama]",2019-07-26,83.0,"[white miniskirt, visible midriff, nagging wif...",90000000.0,41082018.0,...,161.0,ColumbiaPictures,"[English, Italian, Spanish, German]","[USA, UK, China]",Quentin Tarantino,[Quentin Tarantino],"[Leonardo DiCaprio, Brad Pitt, Margot Robbie]",/name/nm0000233/,[/name/nm0000233/],"[/name/nm0000138/, /name/nm0000093/, /name/nm3..."
14526,Monster Hunter,5.3,3835.0,PG-13,"[Action, Adventure, Fantasy]",2020-12-18,NaN,"[based on video game, two word title, parallel...",60000000.0,2201269.0,...,99.0,ConstantinFilm,[English],"[China, Germany, Japan, USA]",Paul W.S. Anderson,"[Paul W.S. Anderson, Kaname Fujioka]","[Milla Jovovich, Tony Jaa, T.I.]",/name/nm0027271/,"[/name/nm0027271/, /name/nm1724522/]","[/name/nm0000170/, /name/nm1388074/, /name/nm1..."
14527,Run,6.7,23944.0,PG-13,"[Horror, Mystery, Thriller]",2020-11-20,67.0,"[mother daughter relationship, villainess, fem...",NaN,NaN,...,90.0,Lionsgate,[English],"[USA, Canada]",Aneesh Chaganty,"[Aneesh Chaganty, Sev Ohanian]","[Sarah Paulson, Kiera Allen, Sara Sohn]",/name/nm3792134/,"[/name/nm3792134/, /name/nm3539578/]","[/name/nm0005299/, /name/nm6266674/, /name/nm3..."
14528,Captain Marvel,6.9,439367.0,PG-13,"[Action, Adventure, Sci-Fi]",2019-03-08,64.0,"[cat, super powers, supernatural power, based ...",160000000.0,153433423.0,...,123.0,WaltDisneyPictures,[English],"[USA, Australia]",Anna Boden,"[Anna Boden, Ryan Fleck, 6 more credits]","[Brie Larson, Samuel L. Jackson, Ben Mendelsohn]",/name/nm1349818/,"[/name/nm1349818/, /name/nm0281396/, fullcredi...","[/name/nm0488953/, /name/nm0000168/, /name/nm0..."


In [42]:
# Save the data to pickle file
movie_df.to_pickle('../dump/movie_data_general_all')

### Simple data visualization

In [ ]:
# Test loading data from dump
movie_df = pd.read_pickle('../dump/movie_data_general2')

In [ ]:
movie_df

In [ ]:
# Check years
movie_df.release_date.dt.year.nunique()
# np.sort(movie_detail_df.release_date.dt.year.unique())

In [ ]:
movie_df.columns

In [ ]:
x = movie_df['release_date'].dt.year
y = movie_df['rating']
fig, ax = plt.subplots(1,1)
plt.scatter(x,y,alpha=0.5)
ax.set_title('',fontsize=15)
ax.set_xlabel('Year',fontsize=15)
ax.set_ylabel('Rating',fontsize=15)
ax.set_yticks([3,4,5,6,7,8,9])
ax.set_yticklabels(['3.0','4.0','5.0','6.0','7.0','8.0','9.0'])
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)

In [ ]:
movie_df.dropna(subset=['gross_usa'])
x = movie_df['gross_usa']
y = movie_df['rating']
fig, ax = plt.subplots(1,1)
plt.scatter(x,y,alpha=0.5)
ax.set_title('',fontsize=15)
ax.set_xlabel('Gross World($)',fontsize=15)
ax.set_ylabel('Rating',fontsize=15)
ax.set_xticks([100e6,200e6,300e6,400e6])
ax.set_xticklabels(['100M','200M','300M','400M'])
ax.set_yticks([3,4,5,6,7,8,9])
ax.set_yticklabels(['3.0','4.0','5.0','6.0','7.0','8.0','9.0'])
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
movie_df.shape

In [ ]:
movie_df.dropna(subset=['budget'])
x = movie_df[movie_df.budget < 1e8]['gross_usa']
y = movie_df[movie_df.budget < 1e8]['budget']
fig, ax = plt.subplots(1,1)
plt.scatter(x,y,alpha=0.5)
ax.set_title('',fontsize=15)
ax.set_xlabel('Gross USA ($)',fontsize=15)
ax.set_ylabel('Rating',fontsize=15)
# ax.set_xticks([100e6,200e6,300e6,400e6])
# ax.set_xticklabels(['100M','200M','300M','400M'])
ax.set_yticks([3,4,5,6,7,8,9])
ax.set_yticklabels(['3.0','4.0','5.0','6.0','7.0','8.0','9.0'])
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)

In [ ]:
movie_df.dropna(subset=['vote'])
x = movie_df['vote']
y = movie_df['rating']

fig, ax = plt.subplots(1,1)
plt.scatter(x,y,alpha=0.5)
ax.set_title('',fontsize=15)
ax.set_xlabel('Vote (count)',fontsize=15)
ax.set_ylabel('Rating',fontsize=15)
ax.set_xticks([0,100e3,200e3,300e3,400e3,500e3,600e3,700e3,800e3])
ax.set_xticklabels(['0','100k','200k','300k','400k','500k','600k','700k','800k',])
ax.set_yticks([3,4,5,6,7,8,9])
ax.set_yticklabels(['3.0','4.0','5.0','6.0','7.0','8.0','9.0'])
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)

In [ ]:
movie_df.dropna(subset=['runtime'])
movie_df = movie_df[movie_df.runtime <300]
x = movie_df['runtime']
y = movie_df['rating']

fig, ax = plt.subplots(1,1)
plt.scatter(x,y,alpha=0.5)
ax.set_title('',fontsize=15)
ax.set_xlabel('Runtime (min)',fontsize=15)
ax.set_ylabel('Rating',fontsize=15)

ax.set_xticks([50,100,150,100,250])
ax.set_xticklabels(['50','100','150','200','250'])
ax.set_yticks([3,4,5,6,7,8,9])
ax.set_yticklabels(['3.0','4.0','5.0','6.0','7.0','8.0','9.0'])
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)

In [ ]:
movie_df.info()

In [ ]:
movie_df.dropna(subset=['budget'])